In [ ]:

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
from tensorflow.keras.models import load_model
from PIL import Image
from tensorflow.keras.preprocessing import image
import json

# Load the dataset with TensorFlow Datasets.
dataset, info=tfds.load('oxford_flowers102', with_info=True, as_supervised=True)
train_ds= dataset['train']
test_ds= dataset['test']

# Create a training set and validation set
val_size=int(0.2 * info.splits['train'].num_examples)
train_size=info.splits['train'].num_examples-val_size

train_val_ds = train_ds
train_ds=train_val_ds.take(train_size)
val_ds=train_val_ds.skip(train_size)

num_train_examples = info.splits['train'].num_examples
print(f'Number of training examples: {num_train_examples}')

num_test_examples = info.splits['test'].num_examples
print(f'Number of test examples: {num_test_examples}')

# Get the number of classes in the dataset from the dataset info.
num_classes=info.features['label'].num_classes
print(f'Number of classes is: {num_classes}')

# Get the first 3 images and labels
for i, (image_, label) in enumerate(train_ds.take(3)):
    print(f'Image {i+1} shape: {image_.shape}, Label: {label.numpy()}')
    plt.imshow(image_.numpy().astype("uint8"))
    plt.title(f'Label: {label.numpy()}')
    plt.axis('off')
    plt.show()

# Sample image display
plt.imshow(image_.numpy().astype('uint8'))
plt.title(f'Label: {label.numpy()}')
plt.axis('off')
plt.show()

# Load class names
with open('label_map.json', 'r') as f:
    class_names = json.load(f)

sample_image, sample_label = next(iter(train_ds))
class_name = class_names[str(sample_label.numpy())]

plt.imshow(sample_image.numpy())
plt.axis('off')
plt.title(class_name)
plt.show()

# Preprocess function

def preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array /= 255.0
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

# Model definition
model = models.Sequential([
    layers.Flatten(input_shape=(224, 224, 3)),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(train_ds.batch(64), validation_data=val_ds.batch(64), epochs=10)

# Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')
plt.show()

# Plot training and validation loss
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')
plt.show()

# Evaluate on test set
test_loss, test_accuracy = model.evaluate(test_ds.batch(64))
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')

# Save model
model.save('flower_classifier.keras')
